# DAIN

Based on the [original Colab file](https://github.com/baowenbo/DAIN/issues/44) by btahir. 
and [Alpha](https://github.com/AlphaGit)

In [ ]:
# Clone DAIN sources
# TODO Bring back original repo
#!git clone https://github.com/baowenbo/DAIN /content/DAIN
!git clone -b colab-compatibility --depth 1 https://github.com/AlphaGit/DAIN

In [35]:
! rm -R DAIN/frames

In [36]:
! mkdir DAIN/frames
! mkdir DAIN/frames/input_frames && mkdir DAIN/frames/output_frames

In [39]:
################# Required Configurations ############################

#@markdown # Required Configuration
#@markdown Use the values in here to configure what you'd like DAIN to do.

#@markdown ## Input file
#@markdown Path (relative to the root of your Google Drive) to the input file. For instance, if you save your `example.mkv` file in your Google Drive, inside a `videos` folder, the path would be: `videos/example.mkv`. Currenly videos and gifs are supported.
INPUT_FILEPATH = "video/original.mp4" #@param{type:"string"}

#@markdown ## Output file
#@markdown Output file path: path (relative to the root of your Google Drive) for the output file. It will also determine the filetype in the destination. `.mp4` is recommended for video input, `.gif` for gif inputs.
OUTPUT_FILE_PATH = "video/output.mp4" #@param{type:"string"}

################# Optional configurations ############################

#@markdown # Optional Configuration
#@markdown Parameters below can be left with their defaults, but feel free to adapt them to your needs.

#@markdown ## Target FPS
#@markdown  how many frames per second should the result have. This will determine how many intermediate images are interpolated.
TARGET_FPS = 60 #@param{type:"number"}

#@markdown ## Frame input directory
#@markdown A path, relative to your GDrive root, where you already have the list of frames in the format 00001.png, 00002.png, etc.
FRAME_INPUT_DIR = 'frames/input_frames' #@param{type:"string"}

#@markdown ## Frame output directory
#@markdown A path, relative to your GDrive root, where you want the generated frame.
FRAME_OUTPUT_DIR = 'frames/output_frames' #@param{type:"string"}

#@markdown ## Seamless playback
#@markdown Creates a seamless loop by using the first frame as last one as well. Set this to True this if loop is intended.
SEAMLESS = False #@param{type:"boolean"}

#@markdown ## Resize hotfix
#@markdown DAIN frames are a bit "shifted / smaller" compared to original input frames. This can partly be mitigated with resizing DAIN frames to the resolution +2px and cropping the result to the original resoultion with the starting point (1,1). Without this fix, DAIN tends to make "vibrating" output and it is pretty noticible with static elements like text.
#@markdown
#@markdown This hotfix tries to make such effects less visible for a smoother video playback. I do not know what DAINAPP uses as a fix for this problem, but the original does show such behaviour with the default test images. More advanced users can change the interpolation method. The methods cv2.INTER_CUBIC and cv2.INTER_LANCZOS4 are recommended. The current default value is cv2.INTER_LANCZOS4.
RESIZE_HOTFIX = True #@param{type:"boolean"}

#@markdown ## Auto-remove PNG directory
#@markdown Auto-delete output PNG dir after ffmpeg video creation. Set this to `False` if you want to keep the PNG files.
AUTO_REMOVE = True #@param{type:"boolean"}

In [38]:
# Check your current GPU
# If you are lucky, you get 16GB VRAM. If you are not lucky, you get less. VRAM is important. The more VRAM, the higher the maximum resolution will go.

# 16GB: Can handle 720p. 1080p will procude an out-of-memory error. 
# 8GB: Can handle 480p. 720p will produce an out-of-memory error.

!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
Quadro P5000, 450.36.06, 16278 MiB


# Install dependencies.

This next step may take somewhere between 15-20 minutes. Run this only once at startup.

Look for the "Finished installing dependencies"  message.

In [ ]:
!apt-get update

In [ ]:
!apt-get install imagemagick imagemagick-doc libgl1-mesa-glx ffmpeg

In [ ]:
!pip install --upgrade pip

In [ ]:
# Install known used versions of PyTorch and SciPy
!pip install torch==1.4.0+cu100 torchvision==0.5.0+cu100 -f https://download.pytorch.org/whl/torch_stable.html
!pip install scipy==1.1.0

In [ ]:
!pip install opencv-python
!pip install imageio

In [ ]:
# This takes a while. Just wait. ~15 minutes.
# Building DAIN.
%cd DAIN/my_package/
!./build.sh
print("Building #1 done.")
%cd ../../

In [ ]:
# Wait again. ~5 minutes.
# Building DAIN PyTorch correlation package.
%cd DAIN/PWCNet/correlation_package_pytorch1_0
!./build.sh
print("Building #2 done.")
%cd ../../..

In [ ]:
# Downloading pre-trained model
!mkdir DAIN/model_weights
!wget -O DAIN/model_weights/best.pth http://vllab1.ucmerced.edu/~wenbobao/DAIN/best.pth

In [ ]:
!CUDA_VISIBLE_DEVICES=0
print("Finished installing dependencies.")

## Start

In [44]:
import os
import cv2

In [45]:
# Detecting FPS of input file.

filename = os.path.basename(INPUT_FILEPATH)

cap = cv2.VideoCapture(f'video/{filename}')

fps = cap.get(cv2.CAP_PROP_FPS)

if(fps/TARGET_FPS>0.5):
  print("Define a higher fps, because there is not enough time for new frames. (Old FPS)/(New FPS) should be lower than 0.5. Interpolation will fail if you try.")

In [46]:
# ffmpeg extract - Generating individual frame PNGs from the source file.
! ffmpeg -i 'video/{filename}' 'DAIN/{FRAME_INPUT_DIR}/%05d.png'

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

In [47]:
! ls 'DAIN/{FRAME_INPUT_DIR}' | wc -l

1625


In [49]:
%cd DAIN/
from IPython.display import clear_output
clear_output()

pngs_generated_count = 1625

import shutil
if SEAMLESS==True:
  pngs_generated_count += 1
  original = str(FRAME_INPUT_DIR)+"/00001.png"
  target = str(FRAME_INPUT_DIR)+"/"+str(pngs_generated_count).zfill(5)+".png"
  shutil.copyfile(original, target)

print(f"Input FPS: {fps}")
print(f"{pngs_generated_count} frame PNGs generated.")

# Checking if PNG do have alpha
import subprocess as sp
%cd {FRAME_INPUT_DIR}
channels = sp.getoutput('identify -format %[channels] 00001.png')
print (f"{channels} detected")

# Removing alpha if detected
if "a" in channels:
  print("Alpha detected and will be removed.")
  print(sp.getoutput('find . -name "*.png" -exec convert "{}" -alpha off PNG24:"{}" \;'))

%cd ../../

Input FPS: 23.98
1625 frame PNGs generated.
/notebooks/DAIN/frames/input_frames
srgb detected
/notebooks/DAIN


In [50]:
# Interpolation

In [51]:
!python -W ignore colab_interpolate.py --netName DAIN_slowmotion --time_step {fps/TARGET_FPS} --start_frame 1 --end_frame {pngs_generated_count} --frame_input_dir '{FRAME_INPUT_DIR}' --frame_output_dir '{FRAME_OUTPUT_DIR}'

revise the unique id to a random numer 88004
Namespace(SAVED_MODEL=None, alpha=[0.0, 1.0], arg='./model_weights/88004-Wed-Sep-16-01-39/args.txt', batch_size=1, channels=3, ctx_lr_coe=1.0, datasetName='Vimeo_90K_interp', datasetPath='', dataset_split=97, debug=False, depth_lr_coe=0.001, dtype=<class 'torch.cuda.FloatTensor'>, end_frame=1625, epsilon=1e-06, factor=0.2, filter_lr_coe=1.0, filter_size=4, flow_lr_coe=0.01, force=False, frame_input_dir='frames/input_frames', frame_output_dir='frames/output_frames', log='./model_weights/88004-Wed-Sep-16-01-39/log.txt', lr=0.002, netName='DAIN_slowmotion', no_date=False, numEpoch=100, occ_lr_coe=1.0, patience=5, rectify_lr=0.001, save_path='./model_weights/88004-Wed-Sep-16-01-39', save_which=1, seed=1, start_frame=1, time_step=0.39966666666666667, uid=None, use_cuda=True, use_cudnn=1, weight_decay=0, workers=8)
cudnn is used
Interpolate 1 frames
****** Processed frame 1 | Time per frame (avg): 11.03s | Time left: 4:58:32.985567 ***************

In [52]:
# Finding DAIN Frames, upscaling and cropping to match original
import numpy as np
if(RESIZE_HOTFIX==True):
  images = []
  for filename in os.listdir(f'{FRAME_OUTPUT_DIR}'):
    img = cv2.imread(os.path.join(f'{FRAME_OUTPUT_DIR}',filename))
    part_filename = os.path.splitext(filename)
    if(part_filename[0].endswith('0')==False):
      dimension = (img.shape[1]+2, img.shape[0]+2)
      resized = cv2.resize(img, dimension, interpolation=cv2.INTER_LANCZOS4)
      crop = resized[1:(dimension[1]-1), 1:(dimension[0]-1)]
      ns = str(f'{FRAME_OUTPUT_DIR}/')+part_filename[0]+".png"
      print(ns)
      cv2.imwrite(ns, crop)

frames/output_frames/00001001.png
frames/output_frames/00002001.png
frames/output_frames/00003001.png
frames/output_frames/00004001.png
frames/output_frames/00005001.png
frames/output_frames/00006001.png
frames/output_frames/00007001.png
frames/output_frames/00008001.png
frames/output_frames/00009001.png
frames/output_frames/00010001.png
frames/output_frames/00011001.png
frames/output_frames/00012001.png
frames/output_frames/00013001.png
frames/output_frames/00014001.png
frames/output_frames/00015001.png
frames/output_frames/00016001.png
frames/output_frames/00017001.png
frames/output_frames/00018001.png
frames/output_frames/00019001.png
frames/output_frames/00020001.png
frames/output_frames/00021001.png
frames/output_frames/00022001.png
frames/output_frames/00023001.png
frames/output_frames/00024001.png
frames/output_frames/00025001.png
frames/output_frames/00026001.png
frames/output_frames/00027001.png
frames/output_frames/00028001.png
frames/output_frames/00029001.png
frames/output_

In [ ]:
!pip install tqdm

In [53]:
from tqdm.notebook import tqdm

In [57]:
def img2video(read_path,save_path):
    print(read_path,save_path)
    files = []
    for r, _, f in os.walk(read_path):
        for file in f:
            if '.png' in file:
                files.append([os.path.join(r, file),file.replace(".png","")])
    print("")
    files.sort(key=lambda x: int(x[1]))
    img = cv2.imread(files[0][0])
    height, width, _ = img.shape
    size = (width,height)
    print("VidSize: ",str(size))
    print("")
    #-----------------------------------------------
    out = cv2.VideoWriter(save_path.replace(".mp4","")+"_60fps.mp4",cv2.VideoWriter_fourcc(*'mp4v'), 60, size)

    for file in tqdm(range(len(files))):
        path = files[file][0]
        img = cv2.imread(path)
        out.write(img)
    out.release()
    

In [58]:
!ls

AverageMeter.py   __pycache__			 loss_function.py
Colab_DAIN.ipynb  balancedsampler.py		 lr_scheduler.py
LICENSE		  checkpoints			 model_weights
MegaDepth	  colab_interpolate.py		 my_args.py
PWCNet		  datasets			 my_package
README.md	  demo_MiddleBury.py		 networks
Resblock	  demo_MiddleBury_slowmotion.py  train.py
S2D_models	  environment.yaml
Stack.py	  frames


In [59]:
img2video(FRAME_OUTPUT_DIR,'../'+str(OUTPUT_FILE_PATH))

frames/output_frames ../video/output.mp4

VidSize:  (640, 480)



In [60]:
%cd ..

/notebooks
